# Network Metrics Deep Dive

This notebook explores various network metrics to understand organizational structure.

## Topics:
- Centrality measures
- Structural analysis
- Identifying key connectors


In [1]:
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

sys.path.insert(0, str(Path().resolve().parent))

from orgnet.core import OrganizationalNetworkAnalyzer
from orgnet.metrics.centrality import CentralityAnalyzer
from orgnet.metrics.structural import StructuralAnalyzer

print("✓ Imports successful")


✓ Imports successful


In [2]:
# Load data and build graph
analyzer = OrganizationalNetworkAnalyzer()
data_paths = {
    "hris": "data/hris.csv",
    "email": "data/email.csv",
    "slack": "data/slack.csv",
    "calendar": "data/calendar.csv",
}

analyzer.load_data(data_paths)
graph = analyzer.build_graph()

print(f"Graph: {graph.number_of_nodes()} nodes, {graph.number_of_edges()} edges")


Graph: 50 nodes, 400 edges


## Centrality Measures

Centrality measures identify the most important nodes in the network.


In [3]:
# Initialize centrality analyzer
centrality = CentralityAnalyzer(graph)

# Compute all centrality measures
all_centralities = centrality.compute_all_centralities()

print("Available centrality measures:")
for metric_name in all_centralities.keys():
    print(f"  - {metric_name}")


Available centrality measures:
  - degree
  - betweenness
  - eigenvector
  - closeness
  - pagerank


In [4]:
# Degree Centrality - measures direct connections
degree_df = all_centralities['degree']
print("=== Top 10 by Degree Centrality ===")
print(degree_df.head(10)[['node_id', 'degree_weighted']])
print("\nInterpretation: People with high degree have many direct connections")


=== Top 10 by Degree Centrality ===
   node_id  degree_weighted
43  emp044        20.220094
41  emp042        16.457345
11  emp012        13.843680
39  emp040        13.567570
34  emp035        12.730242
42  emp043        12.475002
47  emp048        12.440133
35  emp036        11.828234
8   emp009        11.728532
6   emp007        11.718034

Interpretation: People with high degree have many direct connections


In [5]:
# Betweenness Centrality - measures how often a node lies on shortest paths
betweenness_df = all_centralities['betweenness']
print("=== Top 10 by Betweenness Centrality ===")
print(betweenness_df.head(10))
print("\nInterpretation: High betweenness = key connector/broker in network")


=== Top 10 by Betweenness Centrality ===
   node_id  betweenness_centrality
41  emp042                0.278912
33  emp034                0.243197
45  emp046                0.136905
2   emp003                0.109694
22  emp023                0.104592
46  emp047                0.098639
18  emp019                0.082483
21  emp022                0.080782
40  emp041                0.073129
1   emp002                0.072279

Interpretation: High betweenness = key connector/broker in network


## Structural Analysis

Structural analysis identifies brokers and core-periphery structure.


In [6]:
# Structural analyzer
structural = StructuralAnalyzer(graph)

# Compute constraint (structural holes)
constraint_df = structural.compute_constraint()
print("=== Top 10 Brokers (Low Constraint) ===")
print(constraint_df.head(10))
print("\nInterpretation: Low constraint = more brokerage opportunities")


=== Top 10 Brokers (Low Constraint) ===
   node_id  constraint
35  emp036    0.122421
43  emp044    0.124534
39  emp040    0.126337
47  emp048    0.128561
41  emp042    0.131212
34  emp035    0.136869
46  emp047    0.138380
12  emp013    0.139298
7   emp008    0.140082
5   emp006    0.142534

Interpretation: Low constraint = more brokerage opportunities


In [7]:
# Identify effective brokers (low constraint + high betweenness)
brokers = structural.identify_brokers(
    betweenness_df, 
    constraint_df, 
    top_n=10
)

print("=== Top 10 Effective Brokers ===")
print(brokers[['node_id', 'broker_score', 'betweenness_centrality', 'constraint']])
print("\nInterpretation: These people bridge different parts of the organization")


=== Top 10 Effective Brokers ===
   node_id  broker_score  betweenness_centrality  constraint
0   emp042      0.989751                0.278912    0.131212
1   emp034      0.809905                0.243197    0.219569
2   emp046      0.604263                0.136905    0.199828
5   emp047      0.593589                0.098639    0.138380
3   emp003      0.578879                0.109694    0.171393
6   emp019      0.535486                0.082483    0.158404
7   emp022      0.531255                0.080782    0.158896
10  emp013      0.528494                0.068878    0.139298
11  emp049      0.504587                0.060374    0.144114
8   emp041      0.493274                0.073129    0.177351

Interpretation: These people bridge different parts of the organization
